# モデルのトレーニング

このノートブックは、前処理されたデータを使用してYOLOv8モデルをトレーニングするために設計されています。ハイパーパラメータの設定、トレーニングの進捗状況の監視、トレーニング済みモデルの保存が含まれます。

In [ ]:
# 必要なライブラリをインポート
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from src.config import Config
from src.detection import YOLOv8Model
from src.utils import save_model
from scripts.train import train_model


In [ ]:
# 前処理済みのデータセットをロード
# data.yaml と train.txt を使用してデータセットをロードするように変更します。
# 全フレームが格納された単一の .npy ファイルと、
# `labels/frame_xxxxxx.txt` 形式のアノテーションファイルを処理する
# カスタムデータセットクラス (例: MyCustomDataset) を `src.dataset` などに実装し、
# ここでインポートして使用してください。

# from src.dataset import MyCustomDataset # ユーザー定義のデータセットクラスをインポート
# (torchvision.transforms は既にセル2でインポートされています)

# Config.DATA_YAML_PATH が data.yaml ファイルへのパスを指していると仮定します。
# MyCustomDataset はこの data.yaml を解釈し、
#   - 単一の .npy ファイル (例: all_video_frames.npy) をロード
#   - train.txt (または val.txt) に記載されたフレーム識別子に基づいて、.npy ファイルから該当フレームを抽出
#   - 同様にフレーム識別子に基づいて、対応するアノテーションファイル (labels/frame_xxxxxx.txt) をロード
# するように実装されることを想定しています。

# # 必要な変換を定義 (例)
# data_transforms = transforms.Compose([
#     # transforms.ToTensor(), # .npy からロードしたNumPy配列をTensorに変換
#     # その他、モデルに必要な前処理
# ])

print(f"INFO: {Config.DATA_YAML_PATH if hasattr(Config, 'DATA_YAML_PATH') else 'Config.DATA_YAML_PATH (未設定)'} を参照し、")
print(f"      単一のフレームNPYファイル (例: all_video_frames.npy) と、")
print(f"      train.txt (フレーム識別子のリスト)、および labels/frame_xxxxxx.txt 形式のアノテーションを処理する")
print(f"      カスタムデータセット (例: MyCustomDataset) を実装し、以下の train_dataset と train_loader を初期化してください。")

train_dataset = ...  # TODO: カスタムデータセット (MyCustomDatasetなど) をここで初期化します。
                     # 例: MyCustomDataset(data_yaml_path=Config.DATA_YAML_PATH, mode='train', transforms=data_transforms)
                     # MyCustomDataset内では、all_video_frames.npy をメモリにロードするか、メモリマップモードで開きます。
                     # train.txt の各行がフレーム識別子となり、それを使って .npy 配列内のインデックスと
                     # 対応する labels/frame_xxxxxx.txt を特定する必要があります。

train_loader = ...   # TODO: 上記の train_dataset から DataLoader を初期化します。
                     # 例: DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=...)

# train_dataset と train_loader が正しく初期化されたか確認するコード (オプション)
# if train_dataset is not Ellipsis and train_dataset is not None and hasattr(train_dataset, '__len__'):
#     print(f"トレーニングデータセット サンプル数: {len(train_dataset)}")
# else:
#     print("警告: train_dataset が正しく初期化されていません。上記のTODOを確認してください。")
# if train_loader is Ellipsis or train_loader is None:
#     print("警告: train_loader が正しく初期化されていません。上記のTODOを確認してください。")


## データセットのフォーマット

このモデルのトレーニングに使用されるデータセットは、以下の構造とフォーマットに従う必要があります。

- **設定ファイル (`data.yaml`)**: データセットのルートパス、トレーニング/検証データへのパス、クラス数、クラス名、そして**全フレームが格納された単一のNPYファイル名**などを定義します。
    ```yaml
    # 例: data.yaml
    path: ../data/processed  # データセットのルートディレクトリ (NPYファイルやlabelsフォルダの基準)
    all_frames_npy: all_video_frames.npy # 全フレームが格納されたNPYファイル名 (pathからの相対パス)
    train: train.txt         # トレーニング用フレーム識別子リストファイルへのパス (pathからの相対パス)
    val: val.txt             # 検証用フレーム識別子リストファイルへのパス (オプション、pathからの相対パス)
    
    nc: 3  # クラス数
    names: ['player', 'ball', 'net']  # クラス名
    
    # オプション: ラベルディレクトリの指定 (pathからの相対パス)
    # label_dir: labels 
    ```

- **画像データ (単一 `.npy` ファイル)**:
    - すべての動画から抽出された全フレームが、**単一の `.npy` ファイル** (例: `data.yaml` の `all_frames_npy` で指定された `all_video_frames.npy`) にまとめて保存されています。
    - この `.npy` ファイルは、形状が `(総フレーム数, 高さ, 幅, チャンネル数)` のようなNumPy配列です。
    - カスタムデータセットクラス (`MyCustomDataset`) は、この単一の `.npy` ファイルをロードし、`train.txt` や `val.txt` に基づいて特定のフレーム（配列のスライス）にアクセスする必要があります。

- **トレーニング/検証リストファイル (`train.txt`, `val.txt`)**:
    - これらのファイルには、トレーニングまたは検証に使用する**フレームの識別子**が1行に1つずつ記述されます。
    - この識別子は、単一の `.npy` ファイル内の特定のフレームと、対応するアノテーションファイル (`labels/frame_xxxxxx.txt`) の両方に紐づく必要があります。
    - 例 (`train.txt`):
        ```
        video1_frame_000000  # 元の動画名とフレーム番号を組み合わせた識別子など
        video1_frame_000001
        video2_frame_000000
        # ... または、単に labels/ ディレクトリ内のファイル名 (拡張子なし)
        # frame_000000
        # frame_000001
        ```
    - カスタムデータセットクラスは、これらの識別子を解釈し、以下の2つのマッピングを行う必要があります:
        1.  識別子 → `all_video_frames.npy` 内の該当フレームのインデックス。
            (このマッピング情報は、前処理時に別途保存するか、識別子の命名規則から導出できるようにする必要があります。)
        2.  識別子 → 対応するアノテーションファイルパス (例: `labels/frame_000000.txt`)。

- **アノテーションデータ**:
    - 各フレーム識別子に対応するアノテーションファイル（`.txt`）が、指定されたディレクトリ（例: `data.yaml` の `path` と `label_dir` を組み合わせた場所、または `labels/frame_xxxxxx.txt` のような固定パス構造）に存在する必要があります。
    - 例えば、`train.txt` に `frame_000000` という識別子がある場合、対応するラベルファイルは `../data/processed/labels/frame_000000.txt` のようになります。
    - アノテーションファイルはYOLOフォーマットに従います:
        - 各行は1つのオブジェクトを表します。
        - 各行の形式: `<object-class-id> <x_center> <y_center> <width> <height>`
        - 座標は、画像の幅と高さによって正規化されます（0から1の間の値）。
        - `<object-class-id>` は、`data.yaml` の `names` リストに対応する0から始まる整数インデックスです。

カスタムデータセットクラス (`MyCustomDataset`) は、これらのファイル (`data.yaml`, `all_video_frames.npy`, `train.txt`/`val.txt`, アノテーション `.txt` ファイル群) を適切に読み込み、モデルが利用できる形式 (特定のフレームの画像データとそのアノテーション) をペアで供給するように構成する必要があります。

In [ ]:
# YOLOv8モデルを初期化
model = YOLOv8Model()
model.to(Config.DEVICE)


In [ ]:
# ハイパーパラメータを設定
num_epochs = Config.NUM_EPOCHS
learning_rate = Config.LEARNING_RATE


In [ ]:
# モデルをトレーニング
train_model(model, train_loader, num_epochs, learning_rate)


In [ ]:
# トレーニング済みモデルを保存
save_model(model, 'models/yolov8_trained.pth')
